In [1]:
import numpy as np
import pandas as pd
from time import time
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import cross_val_score

In [2]:
data = pd.read_csv("./data/diabetes.csv")

In [104]:
data.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [12]:
#copying data
data_0 = data

#create new data frame to impute values
data_1 = pd.DataFrame()
data_1["Glucose"] = data["Glucose"]
data_1["BMI"] = data["BMI"]

In [13]:
#replace zero values with mean (best result)
from sklearn.impute import SimpleImputer
imputer = SimpleImputer(missing_values=0, strategy='mean', copy=False)

imputer = imputer.fit_transform(data_1)
data_1 = pd.DataFrame(data_1, columns = ['Glucose', 'BMI'])

In [14]:
#adding modded columns to back to the data
data_0["Glucose"] = data_1["Glucose"]
data_0["BMI"] = data_1["BMI"]

In [90]:
# sklearn RandomizedSearchCV runner
def run_randomsearch(X, y, clf, param_dist, n_iter_search=20, scoring=None, n_jobs=-1, cv=10, verbose=3):
    from time import time
    from sklearn.model_selection import RandomizedSearchCV
    from sklearn.model_selection import cross_val_score

    random_search = RandomizedSearchCV(clf, param_distributions=param_dist, n_iter=n_iter_search, scoring=scoring, n_jobs=n_jobs,cv=cv,verbose=verbose)

    start = time()
    random_search.fit(X, y)

    print(("\nRandomizedSearchCV took {:.2f} seconds for {:d} candidates parameter settings.").format((time() - start), n_iter_search))
    clf = random_search.best_estimator_
    
    return cross_val_score(clf, X, y.values.ravel(), cv=cv), clf

def run_splitter_X_scaler(data, test_size=0.3, random_state = 0, shuffle=False):
    from sklearn.model_selection import train_test_split
    from sklearn.preprocessing import StandardScaler
    
    feature_columns = [x for x in data.columns[0:-1]]
    label_column = [data.columns[-1]]

    X = data[feature_columns]
    y = data[label_column]

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = test_size, 
                                                        random_state = random_state, shuffle=shuffle)
    
    #standardising the data
    
    scaler = StandardScaler().fit(X_train)
    X_train = scaler.transform(X_train)
    X_test = scaler.transform(X_test)
    
    return X_train, X_test, y_train, y_test

In [91]:
#splitting data
X_train, X_test, y_train, y_test = run_splitter_X_scaler(data_0, test_size=0.3, random_state = 0)

In [92]:
from sklearn.decomposition import PCA
# Make an instance of the Model
pca = PCA(n_components=2, svd_solver='randomized')
X_train = pca.fit_transform(X_train)
X_test = pca.transform(X_test)

## Training

**logisticregression**

In [93]:
#logistic regression training
from scipy.stats import uniform
from sklearn.linear_model import LogisticRegression

logistic = LogisticRegression(solver='saga', tol=1e-2, max_iter=200,
                              random_state=0)
lg_params = dict(C=uniform(loc=0, scale=4),
                     penalty=['l2', 'l1'])

lr_randsearch = run_randomsearch(X_train, y_train, clf=logistic, param_dist=lg_params, cv=5, n_iter_search=10)

Fitting 5 folds for each of 10 candidates, totalling 50 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.



RandomizedSearchCV took 4.43 seconds for 10 candidates parameter settings.


[Parallel(n_jobs=-1)]: Done  24 tasks      | elapsed:    4.3s
[Parallel(n_jobs=-1)]: Done  43 out of  50 | elapsed:    4.4s remaining:    0.7s
[Parallel(n_jobs=-1)]: Done  50 out of  50 | elapsed:    4.4s finished
/Users/ostheperson/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


**Decision tree**

In [59]:
## Hyper Parameters
from scipy.stats import randint

dt_params = {"criterion": ["gini", "entropy"],
              "min_samples_split": randint(2, 20),
              "max_depth": randint(1, 20),
              "min_samples_leaf": randint(1, 20),
              "max_leaf_nodes": randint(2, 20)}

In [94]:
from sklearn.tree import DecisionTreeClassifier

dt = DecisionTreeClassifier()
dt_randsearch = run_randomsearch(X_train, y_train, clf=dt, param_dist=dt_params, cv=5, n_iter_search=10)

Fitting 5 folds for each of 10 candidates, totalling 50 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.



RandomizedSearchCV took 0.23 seconds for 10 candidates parameter settings.


[Parallel(n_jobs=-1)]: Done  50 out of  50 | elapsed:    0.2s finished
/Users/ostheperson/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_search.py:814: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


**GradientBoosting**

In [61]:
## Hyper Parameter Optimization

gb_params = {
    "loss":["deviance", "exponential"],
    "learning_rate": [0.01, 0.025, 0.05, 0.075, 0.1, 0.15, 0.2],
    "min_samples_split": np.linspace(0.1, 0.5, 12),
    "min_samples_leaf": np.linspace(0.1, 0.5, 12),
    "max_depth":[3,5,8],
    "max_features":["log2","sqrt"],
    "criterion": ["friedman_mse",  "mae", "mse"],
    "subsample":[0.5, 0.618, 0.8, 0.85, 0.9, 0.95, 1.0],
    "n_estimators":[20]
    }

In [99]:
from sklearn.ensemble import GradientBoostingClassifier

gb = GradientBoostingClassifier()
gb_randsearch, gb_clf = run_randomsearch(X_train, y_train, clf=gb, param_dist=gb_params, cv=20, n_iter_search=20)

Fitting 20 folds for each of 20 candidates, totalling 400 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  24 tasks      | elapsed:    4.9s
[Parallel(n_jobs=-1)]: Done 368 tasks      | elapsed:   11.6s
[Parallel(n_jobs=-1)]: Done 400 out of 400 | elapsed:   11.9s finished
/Users/ostheperson/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_search.py:814: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)
/Users/ostheperson/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/gradient_boosting.py:1450: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)



RandomizedSearchCV took 11.91 seconds for 20 candidates parameter settings.


**Catboost**

In [63]:
from catboost import CatBoostClassifier, Pool, cv

train_pool = Pool(X_train, y_train)

# CatBoost model definition
catboost_model = CatBoostClassifier(iterations=1000,
                                    custom_loss=['Accuracy'],
                                    loss_function='Logloss')

# Fit CatBoost model
catboost_model.fit(train_pool)

# CatBoost accuracy
acc_catboost = round(catboost_model.score(X_train, y_train) * 100, 2)

Learning rate set to 0.0079
0:	learn: 0.6906489	total: 54.2ms	remaining: 54.1s
1:	learn: 0.6880461	total: 58.1ms	remaining: 29s
2:	learn: 0.6854805	total: 60.4ms	remaining: 20.1s
3:	learn: 0.6828339	total: 63.1ms	remaining: 15.7s
4:	learn: 0.6804008	total: 65ms	remaining: 12.9s
5:	learn: 0.6779347	total: 66.2ms	remaining: 11s
6:	learn: 0.6753260	total: 67.4ms	remaining: 9.56s
7:	learn: 0.6730383	total: 68.5ms	remaining: 8.5s
8:	learn: 0.6708256	total: 70ms	remaining: 7.71s
9:	learn: 0.6684923	total: 71.1ms	remaining: 7.04s
10:	learn: 0.6664224	total: 72.3ms	remaining: 6.5s
11:	learn: 0.6642378	total: 73.5ms	remaining: 6.05s
12:	learn: 0.6620550	total: 74.6ms	remaining: 5.67s
13:	learn: 0.6598813	total: 75.8ms	remaining: 5.34s
14:	learn: 0.6580862	total: 77ms	remaining: 5.05s
15:	learn: 0.6557292	total: 78.2ms	remaining: 4.81s
16:	learn: 0.6539327	total: 79.2ms	remaining: 4.58s
17:	learn: 0.6519151	total: 80.3ms	remaining: 4.38s
18:	learn: 0.6496406	total: 81.6ms	remaining: 4.21s
19:	le

218:	learn: 0.5140566	total: 342ms	remaining: 1.22s
219:	learn: 0.5138637	total: 344ms	remaining: 1.22s
220:	learn: 0.5136425	total: 345ms	remaining: 1.22s
221:	learn: 0.5134803	total: 347ms	remaining: 1.21s
222:	learn: 0.5132654	total: 348ms	remaining: 1.21s
223:	learn: 0.5131591	total: 349ms	remaining: 1.21s
224:	learn: 0.5129589	total: 351ms	remaining: 1.21s
225:	learn: 0.5127120	total: 352ms	remaining: 1.21s
226:	learn: 0.5124885	total: 354ms	remaining: 1.2s
227:	learn: 0.5122284	total: 355ms	remaining: 1.2s
228:	learn: 0.5118759	total: 357ms	remaining: 1.2s
229:	learn: 0.5117521	total: 358ms	remaining: 1.2s
230:	learn: 0.5114336	total: 360ms	remaining: 1.2s
231:	learn: 0.5111936	total: 361ms	remaining: 1.2s
232:	learn: 0.5109830	total: 364ms	remaining: 1.2s
233:	learn: 0.5106623	total: 371ms	remaining: 1.21s
234:	learn: 0.5103011	total: 375ms	remaining: 1.22s
235:	learn: 0.5101600	total: 378ms	remaining: 1.22s
236:	learn: 0.5097680	total: 381ms	remaining: 1.23s
237:	learn: 0.50952

396:	learn: 0.4800525	total: 676ms	remaining: 1.03s
397:	learn: 0.4799709	total: 684ms	remaining: 1.03s
398:	learn: 0.4797966	total: 685ms	remaining: 1.03s
399:	learn: 0.4796133	total: 688ms	remaining: 1.03s
400:	learn: 0.4795277	total: 689ms	remaining: 1.03s
401:	learn: 0.4793701	total: 693ms	remaining: 1.03s
402:	learn: 0.4791964	total: 696ms	remaining: 1.03s
403:	learn: 0.4790426	total: 698ms	remaining: 1.03s
404:	learn: 0.4788827	total: 706ms	remaining: 1.04s
405:	learn: 0.4788082	total: 713ms	remaining: 1.04s
406:	learn: 0.4787229	total: 716ms	remaining: 1.04s
407:	learn: 0.4786472	total: 717ms	remaining: 1.04s
408:	learn: 0.4785206	total: 718ms	remaining: 1.04s
409:	learn: 0.4783212	total: 720ms	remaining: 1.04s
410:	learn: 0.4781430	total: 722ms	remaining: 1.03s
411:	learn: 0.4780407	total: 724ms	remaining: 1.03s
412:	learn: 0.4778355	total: 725ms	remaining: 1.03s
413:	learn: 0.4775408	total: 726ms	remaining: 1.03s
414:	learn: 0.4772565	total: 731ms	remaining: 1.03s
415:	learn: 

623:	learn: 0.4495219	total: 1.02s	remaining: 614ms
624:	learn: 0.4493978	total: 1.02s	remaining: 613ms
625:	learn: 0.4492731	total: 1.02s	remaining: 611ms
626:	learn: 0.4491159	total: 1.03s	remaining: 615ms
627:	learn: 0.4489995	total: 1.04s	remaining: 614ms
628:	learn: 0.4489217	total: 1.04s	remaining: 614ms
629:	learn: 0.4488284	total: 1.04s	remaining: 613ms
630:	learn: 0.4486879	total: 1.04s	remaining: 611ms
631:	learn: 0.4485746	total: 1.05s	remaining: 610ms
632:	learn: 0.4484814	total: 1.05s	remaining: 608ms
633:	learn: 0.4483769	total: 1.05s	remaining: 606ms
634:	learn: 0.4482560	total: 1.05s	remaining: 604ms
635:	learn: 0.4481328	total: 1.05s	remaining: 603ms
636:	learn: 0.4480211	total: 1.05s	remaining: 601ms
637:	learn: 0.4479164	total: 1.06s	remaining: 600ms
638:	learn: 0.4477957	total: 1.06s	remaining: 598ms
639:	learn: 0.4475514	total: 1.06s	remaining: 596ms
640:	learn: 0.4474208	total: 1.06s	remaining: 594ms
641:	learn: 0.4472516	total: 1.06s	remaining: 592ms
642:	learn: 

807:	learn: 0.4265345	total: 1.35s	remaining: 321ms
808:	learn: 0.4263732	total: 1.35s	remaining: 320ms
809:	learn: 0.4262092	total: 1.35s	remaining: 318ms
810:	learn: 0.4260668	total: 1.36s	remaining: 317ms
811:	learn: 0.4258773	total: 1.36s	remaining: 315ms
812:	learn: 0.4257512	total: 1.37s	remaining: 315ms
813:	learn: 0.4256450	total: 1.37s	remaining: 314ms
814:	learn: 0.4254091	total: 1.38s	remaining: 312ms
815:	learn: 0.4251859	total: 1.38s	remaining: 311ms
816:	learn: 0.4249631	total: 1.38s	remaining: 309ms
817:	learn: 0.4248718	total: 1.38s	remaining: 307ms
818:	learn: 0.4248090	total: 1.38s	remaining: 305ms
819:	learn: 0.4246660	total: 1.38s	remaining: 304ms
820:	learn: 0.4245472	total: 1.39s	remaining: 302ms
821:	learn: 0.4243902	total: 1.39s	remaining: 300ms
822:	learn: 0.4242285	total: 1.39s	remaining: 299ms
823:	learn: 0.4240847	total: 1.39s	remaining: 297ms
824:	learn: 0.4239579	total: 1.39s	remaining: 295ms
825:	learn: 0.4238455	total: 1.39s	remaining: 294ms
826:	learn: 

In [64]:
# How long will this take?
start_time = time()

# Set params for cross-validation as same as initial model
cv_params = catboost_model.get_params()

# Run the cross-validation for 10-folds (same as the other models)
cv_data = cv(train_pool,
             cv_params,
             fold_count=10,
             )

# How long did it take?
catboost_time = (time() - start_time)

# CatBoost CV results save into a dataframe (cv_data), let's withdraw the maximum accuracy score
acc_cv_catboost = round(np.max(cv_data['test-Accuracy-mean']), 10)

0:	learn: 0.6840359	test: 0.6849834	best: 0.6849834 (0)	total: 59.9ms	remaining: 59.8s
1:	learn: 0.6748428	test: 0.6766102	best: 0.6766102 (1)	total: 111ms	remaining: 55.5s
2:	learn: 0.6663296	test: 0.6688625	best: 0.6688625 (2)	total: 157ms	remaining: 52s
3:	learn: 0.6572886	test: 0.6607535	best: 0.6607535 (3)	total: 200ms	remaining: 49.8s
4:	learn: 0.6494764	test: 0.6539282	best: 0.6539282 (4)	total: 249ms	remaining: 49.6s
5:	learn: 0.6427226	test: 0.6482728	best: 0.6482728 (5)	total: 293ms	remaining: 48.6s
6:	learn: 0.6352488	test: 0.6417747	best: 0.6417747 (6)	total: 334ms	remaining: 47.3s
7:	learn: 0.6286128	test: 0.6359307	best: 0.6359307 (7)	total: 364ms	remaining: 45.2s
8:	learn: 0.6228789	test: 0.6310997	best: 0.6310997 (8)	total: 399ms	remaining: 43.9s
9:	learn: 0.6171357	test: 0.6265217	best: 0.6265217 (9)	total: 430ms	remaining: 42.5s
10:	learn: 0.6120984	test: 0.6223276	best: 0.6223276 (10)	total: 461ms	remaining: 41.5s
11:	learn: 0.6071242	test: 0.6180577	best: 0.6180577 

96:	learn: 0.4801962	test: 0.5589840	best: 0.5569121 (68)	total: 4.91s	remaining: 45.7s
97:	learn: 0.4795026	test: 0.5592022	best: 0.5569121 (68)	total: 4.95s	remaining: 45.6s
98:	learn: 0.4790145	test: 0.5591614	best: 0.5569121 (68)	total: 4.99s	remaining: 45.4s
99:	learn: 0.4780175	test: 0.5587319	best: 0.5569121 (68)	total: 5.02s	remaining: 45.2s
100:	learn: 0.4773501	test: 0.5587522	best: 0.5569121 (68)	total: 5.05s	remaining: 45s
101:	learn: 0.4767099	test: 0.5587248	best: 0.5569121 (68)	total: 5.09s	remaining: 44.8s
102:	learn: 0.4759324	test: 0.5590559	best: 0.5569121 (68)	total: 5.12s	remaining: 44.6s
103:	learn: 0.4754580	test: 0.5592008	best: 0.5569121 (68)	total: 5.15s	remaining: 44.4s
104:	learn: 0.4750462	test: 0.5594302	best: 0.5569121 (68)	total: 5.18s	remaining: 44.2s
105:	learn: 0.4743396	test: 0.5597980	best: 0.5569121 (68)	total: 5.22s	remaining: 44s
106:	learn: 0.4736857	test: 0.5599544	best: 0.5569121 (68)	total: 5.25s	remaining: 43.8s
107:	learn: 0.4729156	test: 0

193:	learn: 0.4293811	test: 0.5673572	best: 0.5569121 (68)	total: 9.21s	remaining: 38.3s
194:	learn: 0.4288606	test: 0.5672951	best: 0.5569121 (68)	total: 9.25s	remaining: 38.2s
195:	learn: 0.4283322	test: 0.5674908	best: 0.5569121 (68)	total: 9.28s	remaining: 38.1s
196:	learn: 0.4278165	test: 0.5675799	best: 0.5569121 (68)	total: 9.31s	remaining: 37.9s
197:	learn: 0.4274526	test: 0.5676255	best: 0.5569121 (68)	total: 9.34s	remaining: 37.8s
198:	learn: 0.4268555	test: 0.5678517	best: 0.5569121 (68)	total: 9.37s	remaining: 37.7s
199:	learn: 0.4262815	test: 0.5682239	best: 0.5569121 (68)	total: 9.4s	remaining: 37.6s
200:	learn: 0.4257745	test: 0.5685475	best: 0.5569121 (68)	total: 9.44s	remaining: 37.5s
201:	learn: 0.4253313	test: 0.5688737	best: 0.5569121 (68)	total: 9.47s	remaining: 37.4s
202:	learn: 0.4248019	test: 0.5689392	best: 0.5569121 (68)	total: 9.51s	remaining: 37.3s
203:	learn: 0.4242297	test: 0.5687276	best: 0.5569121 (68)	total: 9.54s	remaining: 37.2s
204:	learn: 0.4232745	

291:	learn: 0.3735642	test: 0.5789393	best: 0.5569121 (68)	total: 12.7s	remaining: 30.8s
292:	learn: 0.3728511	test: 0.5792908	best: 0.5569121 (68)	total: 12.8s	remaining: 30.8s
293:	learn: 0.3723707	test: 0.5792092	best: 0.5569121 (68)	total: 12.8s	remaining: 30.8s
294:	learn: 0.3717412	test: 0.5787798	best: 0.5569121 (68)	total: 12.8s	remaining: 30.7s
295:	learn: 0.3710716	test: 0.5790001	best: 0.5569121 (68)	total: 12.9s	remaining: 30.6s
296:	learn: 0.3705752	test: 0.5789910	best: 0.5569121 (68)	total: 12.9s	remaining: 30.6s
297:	learn: 0.3699875	test: 0.5789890	best: 0.5569121 (68)	total: 13s	remaining: 30.6s
298:	learn: 0.3695341	test: 0.5791485	best: 0.5569121 (68)	total: 13s	remaining: 30.5s
299:	learn: 0.3689775	test: 0.5791629	best: 0.5569121 (68)	total: 13.1s	remaining: 30.5s
300:	learn: 0.3684333	test: 0.5793699	best: 0.5569121 (68)	total: 13.1s	remaining: 30.4s
301:	learn: 0.3679475	test: 0.5798145	best: 0.5569121 (68)	total: 13.1s	remaining: 30.3s
302:	learn: 0.3674524	tes

389:	learn: 0.3242865	test: 0.5873589	best: 0.5569121 (68)	total: 16.2s	remaining: 25.3s
390:	learn: 0.3239182	test: 0.5876823	best: 0.5569121 (68)	total: 16.2s	remaining: 25.3s
391:	learn: 0.3236046	test: 0.5877687	best: 0.5569121 (68)	total: 16.3s	remaining: 25.3s
392:	learn: 0.3231055	test: 0.5877741	best: 0.5569121 (68)	total: 16.3s	remaining: 25.2s
393:	learn: 0.3225006	test: 0.5880263	best: 0.5569121 (68)	total: 16.4s	remaining: 25.2s
394:	learn: 0.3219833	test: 0.5877018	best: 0.5569121 (68)	total: 16.4s	remaining: 25.1s
395:	learn: 0.3216225	test: 0.5879236	best: 0.5569121 (68)	total: 16.4s	remaining: 25.1s
396:	learn: 0.3211926	test: 0.5880999	best: 0.5569121 (68)	total: 16.5s	remaining: 25s
397:	learn: 0.3208503	test: 0.5883046	best: 0.5569121 (68)	total: 16.5s	remaining: 25s
398:	learn: 0.3204370	test: 0.5884365	best: 0.5569121 (68)	total: 16.5s	remaining: 24.9s
399:	learn: 0.3198764	test: 0.5884052	best: 0.5569121 (68)	total: 16.6s	remaining: 24.9s
400:	learn: 0.3195114	tes

...

576:	learn: 0.2551045	test: 0.6099660	best: 0.5569121 (68)	total: 23s	remaining: 16.9s
577:	learn: 0.2548171	test: 0.6101546	best: 0.5569121 (68)	total: 23.1s	remaining: 16.8s
578:	learn: 0.2545098	test: 0.6103518	best: 0.5569121 (68)	total: 23.1s	remaining: 16.8s
579:	learn: 0.2542390	test: 0.6105113	best: 0.5569121 (68)	total: 23.1s	remaining: 16.8s
580:	learn: 0.2539376	test: 0.6105805	best: 0.5569121 (68)	total: 23.2s	remaining: 16.7s
581:	learn: 0.2537290	test: 0.6105317	best: 0.5569121 (68)	total: 23.2s	remaining: 16.7s
582:	learn: 0.2535022	test: 0.6105573	best: 0.5569121 (68)	total: 23.2s	remaining: 16.6s
583:	learn: 0.2532235	test: 0.6106088	best: 0.5569121 (68)	total: 23.3s	remaining: 16.6s
584:	learn: 0.2529422	test: 0.6108476	best: 0.5569121 (68)	total: 23.3s	remaining: 16.5s
585:	learn: 0.2526766	test: 0.6109380	best: 0.5569121 (68)	total: 23.3s	remaining: 16.5s
586:	learn: 0.2523271	test: 0.6113797	best: 0.5569121 (68)	total: 23.4s	remaining: 16.5s
587:	learn: 0.2520140	t

670:	learn: 0.2295266	test: 0.6233977	best: 0.5569121 (68)	total: 27.8s	remaining: 13.6s
671:	learn: 0.2293228	test: 0.6234177	best: 0.5569121 (68)	total: 27.8s	remaining: 13.6s
672:	learn: 0.2290939	test: 0.6235517	best: 0.5569121 (68)	total: 27.8s	remaining: 13.5s
673:	learn: 0.2288242	test: 0.6236145	best: 0.5569121 (68)	total: 27.9s	remaining: 13.5s
674:	learn: 0.2285927	test: 0.6236032	best: 0.5569121 (68)	total: 27.9s	remaining: 13.4s
675:	learn: 0.2282655	test: 0.6239241	best: 0.5569121 (68)	total: 27.9s	remaining: 13.4s
676:	learn: 0.2280795	test: 0.6241018	best: 0.5569121 (68)	total: 28s	remaining: 13.4s
677:	learn: 0.2278182	test: 0.6244879	best: 0.5569121 (68)	total: 28s	remaining: 13.3s
678:	learn: 0.2276601	test: 0.6247168	best: 0.5569121 (68)	total: 28.1s	remaining: 13.3s
679:	learn: 0.2273047	test: 0.6249855	best: 0.5569121 (68)	total: 28.1s	remaining: 13.2s
680:	learn: 0.2270055	test: 0.6252070	best: 0.5569121 (68)	total: 28.1s	remaining: 13.2s
681:	learn: 0.2267478	tes

763:	learn: 0.2074657	test: 0.6344547	best: 0.5569121 (68)	total: 32.8s	remaining: 10.1s
764:	learn: 0.2072402	test: 0.6349086	best: 0.5569121 (68)	total: 32.9s	remaining: 10.1s
765:	learn: 0.2070471	test: 0.6350030	best: 0.5569121 (68)	total: 33s	remaining: 10.1s
766:	learn: 0.2068389	test: 0.6350733	best: 0.5569121 (68)	total: 33s	remaining: 10s
767:	learn: 0.2065962	test: 0.6353916	best: 0.5569121 (68)	total: 33.2s	remaining: 10s
768:	learn: 0.2063375	test: 0.6354231	best: 0.5569121 (68)	total: 33.3s	remaining: 10s
769:	learn: 0.2061403	test: 0.6357698	best: 0.5569121 (68)	total: 33.4s	remaining: 9.97s
770:	learn: 0.2059620	test: 0.6359682	best: 0.5569121 (68)	total: 33.4s	remaining: 9.93s
771:	learn: 0.2058034	test: 0.6361305	best: 0.5569121 (68)	total: 33.5s	remaining: 9.9s
772:	learn: 0.2055706	test: 0.6362673	best: 0.5569121 (68)	total: 33.6s	remaining: 9.86s
773:	learn: 0.2053184	test: 0.6363703	best: 0.5569121 (68)	total: 33.6s	remaining: 9.82s
774:	learn: 0.2051445	test: 0.63

858:	learn: 0.1881018	test: 0.6472978	best: 0.5569121 (68)	total: 36.7s	remaining: 6.02s
859:	learn: 0.1879180	test: 0.6472245	best: 0.5569121 (68)	total: 36.7s	remaining: 5.98s
860:	learn: 0.1877596	test: 0.6476449	best: 0.5569121 (68)	total: 36.8s	remaining: 5.94s
861:	learn: 0.1876087	test: 0.6478135	best: 0.5569121 (68)	total: 36.8s	remaining: 5.9s
862:	learn: 0.1874730	test: 0.6478638	best: 0.5569121 (68)	total: 36.9s	remaining: 5.85s
863:	learn: 0.1872487	test: 0.6478016	best: 0.5569121 (68)	total: 36.9s	remaining: 5.81s
864:	learn: 0.1870526	test: 0.6479645	best: 0.5569121 (68)	total: 37s	remaining: 5.77s
865:	learn: 0.1869253	test: 0.6480805	best: 0.5569121 (68)	total: 37s	remaining: 5.72s
866:	learn: 0.1867358	test: 0.6483025	best: 0.5569121 (68)	total: 37.3s	remaining: 5.73s
867:	learn: 0.1865132	test: 0.6484084	best: 0.5569121 (68)	total: 37.5s	remaining: 5.71s
868:	learn: 0.1863436	test: 0.6485322	best: 0.5569121 (68)	total: 37.7s	remaining: 5.68s
869:	learn: 0.1861619	test

952:	learn: 0.1718304	test: 0.6580245	best: 0.5569121 (68)	total: 41.1s	remaining: 2.03s
953:	learn: 0.1716403	test: 0.6582884	best: 0.5569121 (68)	total: 41.2s	remaining: 1.98s
954:	learn: 0.1715151	test: 0.6584570	best: 0.5569121 (68)	total: 41.2s	remaining: 1.94s
955:	learn: 0.1713460	test: 0.6584222	best: 0.5569121 (68)	total: 41.2s	remaining: 1.9s
956:	learn: 0.1711822	test: 0.6585825	best: 0.5569121 (68)	total: 41.3s	remaining: 1.85s
957:	learn: 0.1710225	test: 0.6588570	best: 0.5569121 (68)	total: 41.3s	remaining: 1.81s
958:	learn: 0.1708625	test: 0.6590594	best: 0.5569121 (68)	total: 41.4s	remaining: 1.77s
959:	learn: 0.1707142	test: 0.6590334	best: 0.5569121 (68)	total: 41.4s	remaining: 1.72s
960:	learn: 0.1705645	test: 0.6590494	best: 0.5569121 (68)	total: 41.4s	remaining: 1.68s
961:	learn: 0.1704511	test: 0.6593512	best: 0.5569121 (68)	total: 41.5s	remaining: 1.64s
962:	learn: 0.1703305	test: 0.6595046	best: 0.5569121 (68)	total: 41.5s	remaining: 1.59s
963:	learn: 0.1701793	

In [65]:
cat_params = {
    'learning_rate': [0.05, 0.07, 0.09, 0.3],
    'depth': [5, 6, 7],
    'l2_leaf_reg': [1, 3, 5, 7, 9],
    'grow_policy': ['SymmetricTree', 'Depthwise', 'Lossguide']
}

In [97]:
model = CatBoostClassifier()
cat_lr_randsearch = run_randomsearch(X_train, y_train, clf=model, param_dist=cat_params, cv=5, n_iter_search=10)

Fitting 5 folds for each of 10 candidates, totalling 50 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  24 tasks      | elapsed:  1.3min
[Parallel(n_jobs=-1)]: Done  50 out of  50 | elapsed:  3.0min finished


0:	learn: 0.6785801	total: 1.69ms	remaining: 1.69s
1:	learn: 0.6642589	total: 3.58ms	remaining: 1.79s
2:	learn: 0.6538978	total: 5.15ms	remaining: 1.71s
3:	learn: 0.6404398	total: 6.54ms	remaining: 1.63s
4:	learn: 0.6293673	total: 7.6ms	remaining: 1.51s
5:	learn: 0.6198564	total: 8.66ms	remaining: 1.43s
6:	learn: 0.6108232	total: 9.71ms	remaining: 1.38s
7:	learn: 0.6035581	total: 10.8ms	remaining: 1.34s
8:	learn: 0.5973025	total: 11.9ms	remaining: 1.31s
9:	learn: 0.5914986	total: 12.9ms	remaining: 1.28s
10:	learn: 0.5855891	total: 14ms	remaining: 1.26s
11:	learn: 0.5805637	total: 15.1ms	remaining: 1.24s
12:	learn: 0.5753598	total: 16.2ms	remaining: 1.23s
13:	learn: 0.5709774	total: 17.4ms	remaining: 1.22s
14:	learn: 0.5665449	total: 18.5ms	remaining: 1.22s
15:	learn: 0.5636079	total: 19.6ms	remaining: 1.21s
16:	learn: 0.5611535	total: 20.6ms	remaining: 1.19s
17:	learn: 0.5583638	total: 24ms	remaining: 1.31s
18:	learn: 0.5559251	total: 25.2ms	remaining: 1.3s
19:	learn: 0.5538781	total: 

246:	learn: 0.4031722	total: 345ms	remaining: 1.05s
247:	learn: 0.4026236	total: 347ms	remaining: 1.05s
248:	learn: 0.4023157	total: 349ms	remaining: 1.05s
249:	learn: 0.4020256	total: 350ms	remaining: 1.05s
250:	learn: 0.4018326	total: 351ms	remaining: 1.05s
251:	learn: 0.4008894	total: 352ms	remaining: 1.04s
252:	learn: 0.4005081	total: 353ms	remaining: 1.04s
253:	learn: 0.3999697	total: 354ms	remaining: 1.04s
254:	learn: 0.3994283	total: 356ms	remaining: 1.04s
255:	learn: 0.3987987	total: 357ms	remaining: 1.04s
256:	learn: 0.3985983	total: 358ms	remaining: 1.03s
257:	learn: 0.3983957	total: 359ms	remaining: 1.03s
258:	learn: 0.3982334	total: 360ms	remaining: 1.03s
259:	learn: 0.3977131	total: 361ms	remaining: 1.03s
260:	learn: 0.3972507	total: 363ms	remaining: 1.03s
261:	learn: 0.3963956	total: 364ms	remaining: 1.02s
262:	learn: 0.3957855	total: 365ms	remaining: 1.02s
263:	learn: 0.3955813	total: 366ms	remaining: 1.02s
264:	learn: 0.3951706	total: 367ms	remaining: 1.02s
265:	learn: 

483:	learn: 0.3148126	total: 672ms	remaining: 716ms
484:	learn: 0.3147131	total: 678ms	remaining: 720ms
485:	learn: 0.3145685	total: 679ms	remaining: 718ms
486:	learn: 0.3144941	total: 680ms	remaining: 717ms
487:	learn: 0.3143947	total: 682ms	remaining: 716ms
488:	learn: 0.3140952	total: 684ms	remaining: 714ms
489:	learn: 0.3138683	total: 685ms	remaining: 713ms
490:	learn: 0.3136317	total: 686ms	remaining: 711ms
491:	learn: 0.3135369	total: 687ms	remaining: 710ms
492:	learn: 0.3131715	total: 689ms	remaining: 708ms
493:	learn: 0.3130111	total: 690ms	remaining: 707ms
494:	learn: 0.3129185	total: 691ms	remaining: 705ms
495:	learn: 0.3128247	total: 692ms	remaining: 704ms
496:	learn: 0.3126865	total: 694ms	remaining: 702ms
497:	learn: 0.3125703	total: 695ms	remaining: 701ms
498:	learn: 0.3122971	total: 696ms	remaining: 699ms
499:	learn: 0.3119859	total: 698ms	remaining: 698ms
500:	learn: 0.3118936	total: 700ms	remaining: 697ms
501:	learn: 0.3117842	total: 701ms	remaining: 696ms
502:	learn: 

...

797:	learn: 0.2433161	total: 1.03s	remaining: 260ms
798:	learn: 0.2431194	total: 1.03s	remaining: 259ms
799:	learn: 0.2430749	total: 1.03s	remaining: 258ms
800:	learn: 0.2430655	total: 1.03s	remaining: 257ms
801:	learn: 0.2430503	total: 1.03s	remaining: 255ms
802:	learn: 0.2430194	total: 1.03s	remaining: 254ms
803:	learn: 0.2428763	total: 1.04s	remaining: 253ms
804:	learn: 0.2424129	total: 1.04s	remaining: 251ms
805:	learn: 0.2421016	total: 1.04s	remaining: 250ms
806:	learn: 0.2417567	total: 1.04s	remaining: 249ms
807:	learn: 0.2417165	total: 1.04s	remaining: 247ms
808:	learn: 0.2415909	total: 1.04s	remaining: 246ms
809:	learn: 0.2412251	total: 1.04s	remaining: 245ms
810:	learn: 0.2409056	total: 1.04s	remaining: 244ms
811:	learn: 0.2408786	total: 1.05s	remaining: 242ms
812:	learn: 0.2406755	total: 1.05s	remaining: 241ms
813:	learn: 0.2402801	total: 1.05s	remaining: 240ms
814:	learn: 0.2400016	total: 1.05s	remaining: 238ms
815:	learn: 0.2398772	total: 1.05s	remaining: 237ms
816:	learn: 

60:	learn: 0.4949688	total: 69.1ms	remaining: 1.06s
61:	learn: 0.4947001	total: 70.5ms	remaining: 1.07s
62:	learn: 0.4942761	total: 71.7ms	remaining: 1.07s
63:	learn: 0.4930054	total: 72.7ms	remaining: 1.06s
64:	learn: 0.4925882	total: 73.8ms	remaining: 1.06s
65:	learn: 0.4919920	total: 74.8ms	remaining: 1.06s
66:	learn: 0.4916211	total: 75.8ms	remaining: 1.05s
67:	learn: 0.4905591	total: 76.7ms	remaining: 1.05s
68:	learn: 0.4892360	total: 77.7ms	remaining: 1.05s
69:	learn: 0.4882538	total: 78.7ms	remaining: 1.04s
70:	learn: 0.4876355	total: 79.7ms	remaining: 1.04s
71:	learn: 0.4871852	total: 80.6ms	remaining: 1.04s
72:	learn: 0.4864218	total: 81.6ms	remaining: 1.04s
73:	learn: 0.4858775	total: 82.6ms	remaining: 1.03s
74:	learn: 0.4851173	total: 85.6ms	remaining: 1.05s
75:	learn: 0.4846822	total: 86.7ms	remaining: 1.05s
76:	learn: 0.4835566	total: 87.8ms	remaining: 1.05s
77:	learn: 0.4831439	total: 89.1ms	remaining: 1.05s
78:	learn: 0.4823358	total: 90.1ms	remaining: 1.05s
79:	learn: 0

346:	learn: 0.3401895	total: 400ms	remaining: 752ms
347:	learn: 0.3397481	total: 401ms	remaining: 751ms
348:	learn: 0.3390355	total: 404ms	remaining: 753ms
349:	learn: 0.3386025	total: 406ms	remaining: 753ms
350:	learn: 0.3381923	total: 407ms	remaining: 753ms
351:	learn: 0.3381312	total: 408ms	remaining: 751ms
352:	learn: 0.3378989	total: 409ms	remaining: 750ms
353:	learn: 0.3374197	total: 410ms	remaining: 748ms
354:	learn: 0.3367921	total: 411ms	remaining: 747ms
355:	learn: 0.3365143	total: 412ms	remaining: 746ms
356:	learn: 0.3361506	total: 413ms	remaining: 744ms
357:	learn: 0.3355387	total: 414ms	remaining: 743ms
358:	learn: 0.3351609	total: 415ms	remaining: 741ms
359:	learn: 0.3350820	total: 417ms	remaining: 741ms
360:	learn: 0.3346579	total: 418ms	remaining: 739ms
361:	learn: 0.3337458	total: 419ms	remaining: 738ms
362:	learn: 0.3330765	total: 420ms	remaining: 736ms
363:	learn: 0.3324171	total: 421ms	remaining: 735ms
364:	learn: 0.3320352	total: 422ms	remaining: 733ms
365:	learn: 

628:	learn: 0.2497215	total: 731ms	remaining: 431ms
629:	learn: 0.2496910	total: 733ms	remaining: 430ms
630:	learn: 0.2495743	total: 735ms	remaining: 430ms
631:	learn: 0.2494792	total: 736ms	remaining: 429ms
632:	learn: 0.2493783	total: 737ms	remaining: 427ms
633:	learn: 0.2488089	total: 738ms	remaining: 426ms
634:	learn: 0.2486309	total: 739ms	remaining: 425ms
635:	learn: 0.2483198	total: 740ms	remaining: 424ms
636:	learn: 0.2482526	total: 741ms	remaining: 422ms
637:	learn: 0.2482220	total: 742ms	remaining: 421ms
638:	learn: 0.2478787	total: 743ms	remaining: 420ms
639:	learn: 0.2477712	total: 744ms	remaining: 419ms
640:	learn: 0.2472749	total: 746ms	remaining: 418ms
641:	learn: 0.2470787	total: 747ms	remaining: 417ms
642:	learn: 0.2468709	total: 749ms	remaining: 416ms
643:	learn: 0.2467925	total: 750ms	remaining: 415ms
644:	learn: 0.2467520	total: 751ms	remaining: 413ms
645:	learn: 0.2466955	total: 752ms	remaining: 412ms
646:	learn: 0.2461374	total: 753ms	remaining: 411ms
647:	learn: 

910:	learn: 0.2009671	total: 1.06s	remaining: 104ms
911:	learn: 0.2007881	total: 1.06s	remaining: 103ms
912:	learn: 0.2007180	total: 1.06s	remaining: 101ms
913:	learn: 0.2006073	total: 1.06s	remaining: 100ms
914:	learn: 0.2003487	total: 1.07s	remaining: 99.1ms
915:	learn: 0.2000932	total: 1.07s	remaining: 97.9ms
916:	learn: 0.1999229	total: 1.07s	remaining: 96.7ms
917:	learn: 0.1997777	total: 1.07s	remaining: 95.5ms
918:	learn: 0.1995364	total: 1.07s	remaining: 94.4ms
919:	learn: 0.1995115	total: 1.07s	remaining: 93.2ms
920:	learn: 0.1994572	total: 1.07s	remaining: 92ms
921:	learn: 0.1992057	total: 1.07s	remaining: 90.8ms
922:	learn: 0.1991608	total: 1.07s	remaining: 89.6ms
923:	learn: 0.1989643	total: 1.07s	remaining: 88.5ms
924:	learn: 0.1987674	total: 1.08s	remaining: 87.3ms
925:	learn: 0.1984661	total: 1.08s	remaining: 86.2ms
926:	learn: 0.1983270	total: 1.08s	remaining: 85ms
927:	learn: 0.1981421	total: 1.08s	remaining: 83.9ms
928:	learn: 0.1978842	total: 1.08s	remaining: 82.7ms
9

In [98]:
#crossvalidation accuracies
print ("Logistic Regression score:: " + str(lr_randsearch[0].mean()))
print ("Decision Tree score:: " + str(dt_randsearch[0].mean()))
print ("Gradient boosting score:: " + str(gb_randsearch[0].mean()))
print ("catboost score:: " +str(cat_lr_randsearch[0].mean()))

Logistic Regression score:: 0.7131948118105762
Decision Tree score:: 0.7019968390185283
Gradient boosting score:: 0.6785714285714286
catboost score:: 0.6854000535538184


## Submission

In [101]:
gb_clf.fit(X_train, y_train)
prediction = gb_clf.predict(X_test)

/Users/ostheperson/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/gradient_boosting.py:1450: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


array([0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1,
       0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0,
       0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,
       1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0,
       0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1,
       1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0,
       1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1,
       0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0])

In [103]:
# Create a submisison dataframe and append the relevant columns
submission = pd.DataFrame()
submission['Outcome'] = prediction # our model predictions on the test dataset
submission.head()

,Outcome
0,0
1,0
2,1
3,1
4,0


In [105]:
# Are our test and submission dataframes the same length?
if len(submission) == len(y_test):
    print("Submission dataframe is the same length as test ({} rows).".format(len(submission)))
else:
    print("Dataframes mismatched, won't be able to submit to Kaggle.")

Submission dataframe is the same length as test (231 rows).


In [106]:
submission.to_csv('../catboost_submission.csv', index=False)
print('Submission CSV is ready!')

Submission CSV is ready!


In [107]:
# Check the submission csv to make sure it's in the right format
submissions_check = pd.read_csv("../catboost_submission.csv")
submissions_check.head()

,Outcome
0,0
1,0
2,1
3,1
4,0
